Load coffea output files and convert them to root files for use with the fitting programs

In [ ]:
from coffea import hist, util
from coffea.processor import accumulate
import numpy as np
import uproot
import os

from ttgamma.utils.plotting import RebinHist, SetRangeHist

In [ ]:
# your timestamps will differ (tip: pressing <Tab> will autocomplete)

outputMC = accumulate(
    [
        util.load("Outputs/output_MCTTGamma_run20211216_114449.coffea"),
        util.load("Outputs/output_MCSingleTop_run20211216_124610.coffea"),
        util.load("Outputs/output_MCTTbar1l_run20211216_121317.coffea"),
        util.load("Outputs/output_MCTTbar2l_run20211216_122542.coffea"),
        util.load("Outputs/output_MCWJets_run20211216_142316.coffea"),
        util.load("Outputs/output_MCZJets_run20211216_130754.coffea"),
        util.load("Outputs/output_MCOther_run20211216_165125.coffea"),
    ]
)

outputData = util.load("Outputs/output_Data_run20211216_171828.coffea")

Get M3 distributions, grouped into top and non-top categories, saving to a root file

In [ ]:
groupingTop = {
    "TopPair": [
        "TTGamma_Dilepton",
        "TTGamma_SingleLept",
        "TTGamma_Hadronic",
        "TTbarPowheg_Dilepton",
        "TTbarPowheg_Semilept",
        "TTbarPowheg_Hadronic",
    ],
    "NonTop": [
        "W1jets",
        "W2jets",
        "W3jets",
        "W4jets",
        "DYjetsM10to50",
        "DYjetsM50",
        "ST_s_channel",
        "ST_tW_channel",
        "ST_tbarW_channel",
        "ST_tbar_channel",
        "ST_t_channel",
        "WGamma",
        "ZGamma_01J_5f_lowMass",
        "TTWtoLNu",
        "TTWtoQQ",
        "TTZtoLL",
        "GJets_HT40To100",
        "GJets_HT100To200",
        "GJets_HT200To400",
        "GJets_HT400To600",
        "GJets_HT600ToInf",
#         "QCD_Pt20to30_Ele",
#         "QCD_Pt30to50_Ele",
#         "QCD_Pt50to80_Ele",
#         "QCD_Pt80to120_Ele",
#         "QCD_Pt120to170_Ele",
#         "QCD_Pt170to300_Ele",
#         "QCD_Pt300toInf_Ele",
#         "QCD_Pt20to30_Mu",
#         "QCD_Pt30to50_Mu",
#         "QCD_Pt50to80_Mu",
#         "QCD_Pt80to120_Mu",
#         "QCD_Pt120to170_Mu",
#         "QCD_Pt170to300_Mu",
#         "QCD_Pt300to470_Mu",
#         "QCD_Pt470to600_Mu",
#         "QCD_Pt600to800_Mu",
#         "QCD_Pt800to1000_Mu",
#         "QCD_Pt1000toInf_Mu",
    ],
}

h = outputMC["M3"].sum("lepFlavor")
h = h.group(
    "dataset", hist.Cat(r"dataset", r"Samples", sorting="placement"), groupingTop
)
h = h.sum("category")
h = RebinHist(h, "M3", 5)
h = SetRangeHist(h, "M3", 50, 550)

hData = outputData["M3"].sum("lepFlavor")
hData = hData.sum("dataset")
hData = hData.sum("category")
hData = hData.sum("systematic")
hData = RebinHist(hData, "M3", 5)
hData = SetRangeHist(hData, "M3", 50, 550)

outdir = "RootFiles"
if not os.path.exists(outdir):
    os.makedirs(outdir)

outputFile = uproot.recreate(os.path.join(outdir, "M3_Output.root"))

outputFile["dataObs"] = hData.to_hist()

datasets = h.axis("dataset").identifiers()
systematics = h.axis("systematic").identifiers()
for _dataset in datasets:
    for _systematic in systematics:
        outputFile[f"{_dataset}_{_systematic}"] = h.integrate("dataset", _dataset).integrate("systematic", _systematic).to_hist()

outputFile.close()


Get photon charged hadron isolation distributions, grouped into isolated and nonprompt categories, saving to a root file

In [ ]:
groupingPho = {
    "Isolated": slice(1, 3),
    "NonPrompt": slice(3, 5),
}

bins = np.array([0, 1.15, 2.5, 4.9, 9, 14.9, 20])

h = outputMC["photon_chIso"].sum("lepFlavor")
h = h.group(
    "category", hist.Cat(r"category", r"Samples", sorting="placement"), groupingPho
)
h = h.sum("dataset")
h = h.rebin("chIso", hist.Bin("chIso", h.axis("chIso").label, bins))

hData = outputData["photon_chIso"].sum("lepFlavor")
hData = hData.sum("dataset")
hData = hData.sum("category")
hData = hData.sum("systematic")
hData = hData.rebin("chIso", hist.Bin("chIso", hData.axis("chIso").label, bins))

outputFile = uproot.recreate(os.path.join(outdir, "Isolation_Output.root"))
outputFile["dataObs"] = hData.to_hist()

categories = h.axis("category").identifiers()
systematics = h.axis("systematic").identifiers()
for _category in categories:
    for _systematic in systematics:
        outputFile[f"{_category}_{_systematic}"] = (
            h.integrate("category", _category)
            .integrate("systematic", _systematic)
            .to_hist()
        )

outputFile.close()

Save distributions of e-gamma mass

In [ ]:
groupingDataset = {
    "WGamma": ["WGamma"],
    "ZGamma": ["ZGamma_01J_5f_lowMass"],
    "Other": [
        "TTGamma_Dilepton",
        "TTGamma_SingleLept",
        "TTGamma_Hadronic",
        "TTbarPowheg_Dilepton",
        "TTbarPowheg_Semilept",
        "TTbarPowheg_Hadronic",
        "W1jets",
        "W2jets",
        "W3jets",
        "W4jets",
        "DYjetsM50",
        "DYjetsM10to50",
        "ST_s_channel",
        "ST_tW_channel",
        "ST_tbarW_channel",
        "ST_tbar_channel",
        "ST_t_channel",
        "TTWtoLNu",
        "TTWtoQQ",
        "TTZtoLL",
    ],
}

groupingPho = {
    "Genuine": slice(1, 2),
    "MisIDele": slice(2, 3),
    "NonPrompt": slice(3, 5),
}


h = outputMC["photon_lepton_mass_3j0t"]
h = h.group(
    "category", hist.Cat(r"category", r"Samples", sorting="placement"), groupingPho
)
h = h.group(
    "dataset", hist.Cat(r"dataset", r"Samples", sorting="placement"), groupingDataset
)
h = RebinHist(h, "mass", 5)
h = SetRangeHist(h, "mass", 40, 200)

hData = outputData["photon_lepton_mass_3j0t"]
hData = hData.sum("dataset")
hData = hData.sum("category")
hData = hData.sum("systematic")
hData = RebinHist(hData, "mass", 5)
hData = SetRangeHist(hData, "mass", 40, 200)


systematics = h.axis("systematic").identifiers()

for _lepton in ["electron", "muon"]:
    outputFile = uproot.recreate(os.path.join(outdir, f"MisID_Output_{_lepton}.root"))

    outputFile["dataObs"] = hData.integrate("lepFlavor", _lepton).to_hist()

    hMisID = (
        h.integrate("category", "MisIDele")
        .sum("dataset")
        .integrate("lepFlavor", _lepton)
    )
    hOther = h.integrate("category", ["Genuine", "NonPrompt"]).integrate(
        "lepFlavor", _lepton
    )
    datasets = hOther.axis("dataset").identifiers()

    for _systematic in systematics:
        outputFile[f"MisIDele_{_systematic}"] = hMisID.integrate(
            "systematic", _systematic
        ).to_hist()
        for _dataset in datasets:
            outputFile[f"{_dataset}_{_systematic}"] = (
                hOther.integrate("dataset", _dataset)
                .integrate("systematic", _systematic)
                .to_hist()
            )

    outputFile.close()